In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt
import sys
import matplotlib.image
from numpy import genfromtxt
dtype = tf.float32

In [16]:
#data loading
print("Reading Data..")
set1 = genfromtxt('set1_834450x7.txt')
set2 = genfromtxt('set2_834450x7.txt')
data = np.concatenate((set1,set2),axis=0)
print('loading data finished!')

#reshaping
print("Data restructuring..")
numtrainLogs = 6
trainSize = 1000
testSize = 200

y = data[:,numtrainLogs]
x = data[:,0:numtrainLogs]
yreshape = y.reshape(33578,50, 1)
xreshape = x.reshape(33578,50, numtrainLogs)
xreshape = xreshape.reshape(33578,50*6)
yreshape = yreshape.reshape(33578,50*1)
train_x = xreshape[0:trainSize,:]
train_y = yreshape[0:trainSize,:]
test_x = xreshape[trainSize:trainSize+testSize, :]
test_y = yreshape[trainSize:trainSize+testSize, :]
print('train_x shape: ', train_x.shape)
print('train_y shape: ', train_y.shape)
print('test_x shape: ', test_x.shape)
print('test_y shape: ', test_y.shape)

Reading Data..
loading data finished!
Data restructuring..
train_x shape:  (1000, 300)
train_y shape:  (1000, 50)
test_x shape:  (200, 300)
test_y shape:  (200, 50)


In [17]:
def weight_variable(shape, name):
    #return tf.get_variable(name, shape=shape, dtype=dtype, initializer=tf.contrib.layers.xavier_initializer())
    initial = tf.truncated_normal(shape, stddev=0.1, dtype=tf.float32)
    return tf.Variable(initial, name=name, dtype=tf.float32)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape, dtype=dtype)
    return tf.Variable(initial, name=name, dtype=dtype)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv_layer(x, r, c, i, o, name):
    W_conv = weight_variable([r, c, i, o], name + '_w')
    b_conv = bias_variable([o], name + '_b')
    return tf.nn.relu(conv2d(x, W_conv) + b_conv)

def inception_layer(x, i, o11, o21, o22, o31, o32, o41, name):    # input, input_layers, output_layers 1 and 2
    conv1x1_1 = conv_layer(x, 1, 1, i, o11, name + 'conv1x1_1')
    conv1x1_2 = conv_layer(x, 1, 1, i, o21, name + 'conv1x1_2')
    conv3x3 = conv_layer(conv1x1_2, 5, 30, o21, o22, name + 'conv3x3')
    conv1x1_3 = conv_layer(x, 1, 1, i, o31, name + 'conv1x1_3')
    conv5x5 = conv_layer(conv1x1_3, 10, 30, o31, o32, name + 'conv5x5')
    max3x3 = max_pool(x, 3,3, 1)
    conv1x1_4 = conv_layer(max3x3, 1, 1, i, o41, name + 'conv1x1_4')
    return tf.nn.relu(tf.concat([conv1x1_1, conv3x3, conv5x5, conv1x1_4], 3))

def max_pool(x, w, h, s):
    return tf.nn.max_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='SAME')

def avg_pool(x, w, h, s):
    return tf.nn.avg_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='SAME')
def avg_pool_final(x, w, h, s):
    return tf.nn.avg_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='VALID')

def fully_connected(X, layers, out, name):
    pre_l = 0
    i = 1
    for l in layers:
        if not pre_l:
            pre_l = l
            continue
        w_h = weight_variable([pre_l, l], name + '_' + str(i) + '_w')
        b_h = bias_variable([l], name + '_' + str(i) + '_b')
        X = tf.nn.relu(tf.matmul(X, w_h) + b_h)
        pre_l = l
        i += 1
    w_h = weight_variable([pre_l, out], name + '_' + str(i) + '_w')
    b_h = bias_variable([out], name + '_' + str(i) + '_b')
    return tf.matmul(X, w_h) + b_h

In [18]:
#Initializations
print("Initializing variables..")
BATCH_SIZE = 10
learning_rate = 1.00e-4
epoch = 0 
epochNum = 2
data_size = len(train_x)
lossTrain = np.zeros((epochNum,1))
lossTest = np.zeros((epochNum,1))
    
#Model creation: placeholders
print("Creating Model...")
X = tf.placeholder(dtype, [None, 300])
Y = tf.placeholder(dtype, [None, 50])
tr = tf.placeholder(dtype)
pr = tf.placeholder(dtype)
    
#convolution
x_ = tf.reshape(X, [-1, 50, 6, 1])     # 50 x 6 x 1
print("x_", x_)
conv = conv_layer(x_, 5, 6, 1, 64, 'conv1')      # 50 x 6 x 64
print("conv1", conv)
conv = conv_layer(conv, 3, 6, 64, 128, 'conv2')   # 50 x 6 x 128
print("conv2", conv)
pool = max_pool(conv, 6, 5, 2)             # 25 x 3 x 128
print("pool", pool)
norm = tf.contrib.layers.batch_norm(pool)
print("norm", norm)
conv = inception_layer(norm, 128, 8, 64, 96, 8, 16, 8, 'inc1')  # 25 x 3 x 128
print("inc1", conv)
conv = inception_layer(conv, 128, 64, 96, 128, 16, 32, 32, 'inc2')  # 25 x 3 x 256
print("inc2", conv)
conv = inception_layer(conv, 256, 160, 112, 224, 24, 64, 64, 'inc3')  # 25 x 3 x 512
print("inc3", conv)
pool = max_pool(conv, 7, 5, 2)             # 13 x 2 x 256
print("pool", pool)
norm = tf.contrib.layers.batch_norm(pool)
print("norm", norm)
conv = inception_layer(norm, 512, 128, 128, 256, 32, 64, 64, 'inc4')  # 13 x 2 x 512
print("inc4", conv)
pool = avg_pool(conv, 7, 5, 2)             # 7 x 1 x 512 
print("pool", pool)
pool = avg_pool_final(pool, 7, 1, 1)             # 1 x 1 x 512 
print("poolfinal", pool)

#FCL and dropout
cnn = tf.reshape(pool, [-1, 512])    #512
print("cnn", cnn)
cnn = tf.nn.dropout(cnn, pr)
print("cnn dropout", cnn)
out_y = fully_connected(cnn, [512], 50, 'fc1')    #50
print("yhat", out_y)
print("y", Y.shape)

#loss function
loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions = out_y, labels = Y)) 

#training 
train_step = tf.train.AdamOptimizer(tr).minimize(loss)

Initializing variables..
Creating Model...
x_ Tensor("Reshape_2:0", shape=(?, 50, 6, 1), dtype=float32)
conv1 Tensor("Relu_30:0", shape=(?, 50, 6, 64), dtype=float32)
conv2 Tensor("Relu_31:0", shape=(?, 50, 6, 128), dtype=float32)
pool Tensor("MaxPool_6:0", shape=(?, 25, 3, 128), dtype=float32)
norm Tensor("BatchNorm_2/FusedBatchNorm:0", shape=(?, 25, 3, 128), dtype=float32)
inc1 Tensor("Relu_38:0", shape=(?, 25, 3, 128), dtype=float32)
inc2 Tensor("Relu_45:0", shape=(?, 25, 3, 256), dtype=float32)
inc3 Tensor("Relu_52:0", shape=(?, 25, 3, 512), dtype=float32)
pool Tensor("MaxPool_10:0", shape=(?, 13, 2, 512), dtype=float32)
norm Tensor("BatchNorm_3/FusedBatchNorm:0", shape=(?, 13, 2, 512), dtype=float32)
inc4 Tensor("Relu_59:0", shape=(?, 13, 2, 512), dtype=float32)
pool Tensor("AvgPool_2:0", shape=(?, 7, 1, 512), dtype=float32)
poolfinal Tensor("AvgPool_3:0", shape=(?, 1, 1, 512), dtype=float32)
cnn Tensor("Reshape_3:0", shape=(?, 512), dtype=float32)
cnn dropout Tensor("dropout_1/mu

In [ ]:
#load trained model
#with tf.Session() as sess:
#    tf.train.Saver().restore(sess, "./model/testmodel.ckpt")
#    lossTrain = sess.run(loss, feed_dict={X: train_x, Y: train_y, pr: 1.0})
#    lossTest = sess.run(loss, feed_dict={X: test_x, Y: test_y, pr: 1.0})
#    yhat = yhat = sess.run(out_y, feed_dict={X: test_x[1:2,:], pr: 1.0})

In [ ]:
#Running the training session
saver = tf.train.Saver()
sess = tf.Session()
with sess.as_default() as sess:
    tf.global_variables_initializer().run()
    print("Training...")
    while epoch < epochNum:
        batchIter = 0; 
        for start in range(0, data_size, BATCH_SIZE):
            batchIter += 1
            end = start + BATCH_SIZE
            sess.run(train_step, feed_dict={X: train_x[start:end], Y: train_y[start:end], tr: learning_rate, pr: 0.45})
            batchPer = 100*batchIter*BATCH_SIZE/data_size
            if round(batchPer) % 25 == 0:
                print('percent of batch: ', batchPer) 
        #print accuracy 
        lossTrain[epoch] = sess.run(loss, feed_dict={X: train_x, Y: train_y, pr: 1.0})
        lossTest[epoch] = sess.run(loss, feed_dict={X: test_x, Y: test_y, pr: 1.0})
        epoch += 1
        print('epoch: ', epoch, ', Train Accuracy: ', lossTrain[epoch],'Test Accuracy:', lossTest[epoch] , ', learning_rate: ', learning_rate)

In [ ]:
#plotting the loss over epoch
plt.title('Loss over Epochs (Test and Training)')
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error Loss')
epochArray = np.array(list(range(1, epochNum+1)))    
plt.plot(epochArray, lossTrain, 'b')
plt.plot(epochArray, lossTest, 'r')
plt.legend(('Train', 'Test'))
plt.show()


In [ ]:
#plotting prediction vs. label Gamma Ray
depthUnit = 0.5    #each data point is half a foot
depthPoint = np.array(list(range(1, 51)))*depthUnit    
yhat = np.transpose(yhat)
y = np.transpose(train_y[0:1,:])
plt.plot(depthPoint, yhat)
plt.plot(depthPoint, y)
plt.title('GR: Prediction Vs. Original')
plt.xlabel('depth, ft ')
plt.ylabel('American Petroleum Institute, API')
plt.legend(('prediction', 'original'))
plt.show()

In [ ]:
save_path = saver.save(sess, "./model/testmodel.ckpt")

In [ ]:
sess.close()